In [8]:
pip install tavily-python

In [ ]:
## Please go to tavily.com and get a free api to continue and add to your secrets as TAVILY_API_KEY

In [2]:
from tavily import TavilyClient
import google.generativeai as genai
from google.colab import userdata

class ChatHistory:
    def __init__(self):
        self.history = []

    def add_message(self, message, search_result, ai_response):
        self.history.append({
            "message": message,
            "search_result": search_result,
            "ai_response": ai_response
        })

    def get_history(self):
        return self.history

def get_completion_from_messages(messages, model_name="gemini-2.0-flash", temperature=0):
    generation_config = {
        "temperature": temperature,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
    }

    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=generation_config,
    )

    chat = model.start_chat()
    current_response = None

    for msg in messages:
        if msg['role'] == 'user':
            current_response = chat.send_message(msg['content'])

    return current_response.text

# Step 1: Setup API keys and clients
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

# Configure APIs
genai.configure(api_key=GOOGLE_API_KEY)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
chat_history = ChatHistory()

def process_query(query):
    # Step 2: Get search results from Tavily with US news focus
    search_results = tavily_client.search(
        query,
        search_depth="advanced",
        include_domains=[
            "nytimes.com",
            "wsj.com",
            "usatoday.com",
            "washingtonpost.com",
            "latimes.com",
            "nypost.com",
            "foxnews.com",
            "cnn.com",
            "nbcnews.com",
            "abcnews.go.com",
            "cbsnews.com",
            "reuters.com",
            "bloomberg.com",
            "apnews.com"
        ],
        max_results=5  # Limit to top 5 most relevant results
    )

    # Step 3: Prepare context for Gemini Pro with US news focus
    context = f"""
    You are a US news specialist. Using only the following search results from major US news sources,
    provide a comprehensive answer that focuses on how this topic has been covered in US media and its impact
    on American audiences. If available, include relevant quotes from US news sources.

    Search Results:
    {search_results}

    Query: {query}

    Instructions:
    1. Only use information from the provided US news sources
    2. Focus on US media perspective and coverage
    3. Include specific US news source citations when possible
    4. If the topic has limited US coverage, acknowledge this fact
    """

    # Step 4: Get AI response using Gemini Pro
    messages = [{'role': 'user', 'content': context}]
    ai_response = get_completion_from_messages(messages)

    # Step 5: Store in chat history
    chat_history.add_message(query, search_results, ai_response)

    return {
        'query': query,
        'search_results': search_results,
        'ai_response': ai_response
    }

# Test the integration
query = "What is deepseek and what impact has had on US"
result = process_query(query)

print("Query:", result['query'])
print("\nAI Response:", result['ai_response'])
print("\nFull History:", chat_history.get_history())

Query: What is deepseek and what impact has had on US

AI Response: DeepSeek is a Chinese AI company whose recent advancements in AI technology, particularly its AI chatbot, have garnered significant attention and concern in the United States. US media coverage focuses on its potential impact on the US tech market, national security, and the broader geopolitical competition in AI development.

**Impact on US Tech Market and Economy:**

*   **Stock Market Volatility:** DeepSeek's emergence has sent "shock waves through the U.S. tech market," according to NBC News, causing "major stock price shifts" (NBC News). CBS News reported that Nvidia shares tumbled 12.5% in early trading, while ASML shed 7.6% following DeepSeek's announcement.
*   **Competition with US AI Companies:** DeepSeek's AI model is seen as potentially rivaling US-based competitors like ChatGPT (CBS News). NBC News notes that DeepSeek's model seemingly performs on par with US competitors but requires less computing power f

In [3]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import gradio as gr

def gradio_process(query: str) -> str:
    """
    Processes the user query using the process_query function.
    """
    try:
        # Using the process_query function defined in the previous cell
        result = process_query(query)
        return result['ai_response']
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create and launch the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# US News Chat AI")
    gr.Markdown("Enter a query about US news topics and get insights from our AI powered by Gemini Pro.")

    with gr.Row():
        input_text = gr.Textbox(
            lines=2,
            placeholder="Enter your query here...",
            label="Your Query"
        )
        output_text = gr.Textbox(
            lines=10,
            label="AI Response"
        )

    submit_btn = gr.Button("Submit")
    submit_btn.click(
        fn=gradio_process,
        inputs=input_text,
        outputs=output_text
    )

    gr.Examples(
        examples=[
            "Who is Lionel Messi and what impact has he had on US soccer since joining Inter Miami?",
            "What are the latest developments in AI regulation in the United States?",
            "How has the US housing market changed in recent months?"
        ],
        inputs=input_text
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8c45a71ffb1f9b5590.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8c45a71ffb1f9b5590.gradio.live
